In [1]:
import ssl
try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    pass
else:
    ssl._create_default_https_context = _create_unverified_https_context

In [2]:
import google_play_scraper
import os
from datetime import datetime as dt
import json
import re

### Get Review utils

In [3]:
def get_review_data(app_id,lang_list=['en'], country_list=['us','in'],
                    sort=google_play_scraper.Sort.NEWEST,
                    count=1000):
    """
    Get review data for specified languages and countries.

    Args:
    lang_list (list): List of language codes.
    country_list (list): List of country codes.
    sort: Sorting criterion for reviews.
    count: Number of reviews to fetch.

    Returns:
    dict: Dictionary containing review data for each country and language combination.
    """
    review_data = {}
    review_data_serl = {}

    for lang in lang_list:
        for country in country_list:
            try:
                result = google_play_scraper.reviews(
                    app_id,
                    lang=lang,
                    country=country,
                    sort=sort,
                    count=count
                )

                review_data_entry = {
                    'country': country,
                    'lang': lang,
                    'review_response': result[0]
                }

                review_data[f"{country}-{lang}"] = review_data_entry
                for i in range(len(review_data_entry['review_response'])):
                  _at = review_data_entry['review_response'][i]['at']
                  _reat = review_data_entry['review_response'][i]['repliedAt']
                  review_data_entry['review_response'][i]['at'] = _at.strftime("%Y-%m-%d %H:%M:%S") if _at is not None else "NaN"
                  review_data_entry['review_response'][i]['repliedAt'] = _reat.strftime("%Y-%m-%d %H:%M:%S") if _reat is not None else "NaN"
                review_data_serl[f"{country}-{lang}"] = review_data_entry

            except Exception as e:
                print(f"Error fetching reviews for {lang} in {country}: {e}")

    return review_data,review_data_serl

### Running the prompts

In [9]:
# Define the app ID or package name of the app you want to fetch reviews for
APP_ID = 'com.google.android.apps.nbu.paisa.user'

In [10]:
# Fetch basic information about the app
loaded_api_result_app = google_play_scraper.app(APP_ID,country='in',lang='en')

# Creeating app name keeping only alphabets and splitting by space
APP_NAME = re.sub(r'[^a-zA-Z ]+', '', loaded_api_result_app['title']).replace(" ","_")

# Print basic information about the app
print("App Title:", loaded_api_result_app['title'])
print("App Developer:", loaded_api_result_app['developer'])

# store the loaded api result
with open(f"./datas/api_result_appdescr_{APP_NAME}.json", 'w') as f:
    json.dump(loaded_api_result_app, f,indent=4)

App Title: Google Pay: Secure UPI payment
App Developer: Google LLC


In [11]:
review_data,review_data_serl = get_review_data(
    app_id = APP_ID,
    lang_list=['en'],
    country_list=['in'],
    sort=google_play_scraper.Sort.MOST_RELEVANT,
    count=10000
)

In [12]:
standard_review_data_serl = []
for cl in review_data_serl:
  for _ in review_data_serl[cl]['review_response']:
    rev = _
    temp = {}
    temp['title'] = "Ticket created from Playstore review "+rev['reviewId']
    temp['text'] = rev['content']
    temp['date'] = rev['at']
    temp['source'] = "playstore"
    temp['metadata'] = rev
    temp['metadata']['cou_lan'] = cl
    temp['app_name']=APP_NAME
    standard_review_data_serl.append(temp)

In [13]:
with open(f"./datas/api_result_reviews_relv_{APP_NAME}_v{0}.json", 'w') as f:
    json.dump(standard_review_data_serl, f,indent=4)